In [2]:
# enerate a recommendation model based on content filtering in python
# At least use the descriptioon field. You can decide whether to combine the decription with other fields. You decide.
# Save this model in a .sav file

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

articlesdf = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/shared_articles.csv')
usersdf = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/users_interactions.csv')

usersdf.head()



,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [3]:
n_ratings = len(usersdf)
n_articles = len(usersdf['contentId'].unique())
n_users = len(usersdf['personId'].unique())

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique contentId's: {n_articles}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per article: {round(n_ratings/n_articles, 2)}")
print(f"Total possible ratings: {n_articles * n_users}")

Number of ratings: 72312
Number of unique contentId's: 2987
Number of unique users: 1895
Average ratings per user: 38.16
Average ratings per article: 24.21
Total possible ratings: 5660365


In [ ]:
print('Min:\t\t', userstriple['contentId'].value_counts().min())
print('Quartile 1:\t', userstriple['contentId'].value_counts().quantile(.25))
print('Median:\t\t', userstriple['contentId'].value_counts().quantile(.5))
print('Quartile 3:\t', userstriple['contentId'].value_counts().quantile(.75))
print('Max:\t\t', userstriple['contentId'].value_counts().max())

VIEW: The user has opened the article. This is the most basic level of interest (set as 1)
LIKE: The user has liked the article. This is better than viewing (set as 2)
FOLLOW: The user chose to be notified on any new comment in the article. (Set as 3)
BOOKMARK: The user has bookmarked the article for easy return in the future. (Set as 4)
COMMENT CREATED: The user wrote a comment for the article. This is the top interaction indicating the highest interest (set as 5)

In [4]:
for row in usersdf.itertuples():
  if (row.eventType == 'VIEW') :
    usersdf.at[row.Index, 'eventType'] = 1
  elif (row.eventType == 'LIKE'):
    usersdf.at[row.Index, 'eventType'] = 2
  elif (row.eventType == 'FOLLOW'):
    usersdf.at[row.Index, 'eventType'] = 3
  elif (row.eventType == 'BOOKMARK'):
    usersdf.at[row.Index, 'eventType'] = 4
  elif (row.eventType == 'COMMENT CREATED'):
    usersdf.at[row.Index, 'eventType'] = 5
  else:
    usersdf.at[row.Index, 'eventType'] = 0


usersdf

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,1,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,1,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,1,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,3,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,1,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
72307,1485190425,2,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR
72308,1485190425,1,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
72309,1485190072,1,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR
72310,1485190434,1,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR


In [5]:
userstriple = usersdf[['personId', 'contentId', 'eventType']]

In [6]:
userstriple

,personId,contentId,eventType
0,-8845298781299428018,-3499919498720038879,1
1,-1032019229384696495,8890720798209849691,1
2,-1130272294246983140,310515487419366995,1
3,344280948527967603,310515487419366995,3
4,-445337111692715325,-7820640624231356730,1
...,...,...,...
72307,-9016528795238256703,-6590819806697898649,2
72308,102305705598210278,-5813211845057621660,1
72309,-9196668942822132778,-1999468346928419252,1
72310,-9016528795238256703,-6590819806697898649,1


In [7]:
value_counts = userstriple['contentId'].value_counts()
keep_list = value_counts[value_counts >= 31]
userstriple = userstriple.loc[userstriple['contentId'].isin(keep_list.index)]
userstriple.shape

(46847, 3)

In [8]:
def create_matrix(df, user, item, rating):
  import numpy as np
  from scipy.sparse import csr_matrix



  # counting number of movies and users
  U = df[user].nunique()  # Number of users for the matrix
  I = df[item].nunique()  # Number of items for the matrix

  # create dictionary that maps user and item IDs to consecutive, integers with no missing values
  # needs all of the to be consecutive so it makes a dictionary directing that
  # Map user and movie IDs to matrix indices
  user_mapper = dict(zip(np.unique(df[user]), list(range(U))))
  item_mapper = dict(zip(np.unique(df[item]), list(range(I))))

  # need the inverse to go back
  # Map matrix indices back to IDs
  user_inv_mapper = dict(zip(list(range(U)), np.unique(df[user])))
  item_inv_mapper = dict(zip(list(range(I)), np.unique(df[item])))


  # Create a list of index values for the csr_matrix for users and movies
  user_index = [user_mapper[i] for i in df[user]]
  item_index = [item_mapper[i] for i in df[item]]

  # Build the final matrix which will look like: (movieId, userId) rating
  X = csr_matrix((df[rating], (item_index, user_index)), shape=(I, U))

  return X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper

In [9]:
userstriple['eventType'] = userstriple['eventType'].astype(int)

<ipython-input-9-f8b69c598ac3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  userstriple['eventType'] = userstriple['eventType'].astype(int)


In [10]:
X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = create_matrix(userstriple, 'personId', 'contentId', 'eventType')

In [11]:
def recommend(itemId, X, item_mapper, item_inv_mapper, k, metric='cosine', messages=True):
  from sklearn.neighbors import NearestNeighbors
  import numpy as np

  rec_ids = []                # Make a list for the recommended item IDs we'll get later
  item = item_mapper[itemId]  # Get the index of the movie ID passed into the function
  item_vector = X[item]       # Get the vector of user ratings for the movie ID passed into the function

  # Fit the clustering algorithm based on the user-item matrix X -- plus 1 gets another one since it will return itself otherwise
  knn = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric).fit(X)

  # Call the trained knn cluster model to return the nearest neighbors of the item_vector passed in
  rec = knn.kneighbors(item_vector.reshape(1,-1), return_distance=True)
  rec_indeces = rec[1][0]     # Parse out the list of indeces of the recommended items
  rec_distances = rec[0][0]   # Parse out the recommendation strength calculated as the distance from the cluster center
  rec_distances = np.delete(rec_distances, 0) # Drop the first number in the list because it is the distance of itemId from itself

  # We need to replace the recommended item indeces with their original item IDs
  for i in range(1, knn.n_neighbors): # n_neighbors is the number of neighbors to return
    rec_ids.append(item_inv_mapper[rec_indeces[i]])

  # It may help to see what this is. The distance list is first and the recommended item indeces are second
  if messages:
    print(f'List of recommended item indeces:\n{rec_indeces}\n')
    print(f'List of recommended item IDs:\n{rec_ids}\n')
    print(f'List of recommended item similarity to selected item:\n{rec_distances}\n')

  # Return two lists: the original item IDs of the recommendations and their similarity scores
  return rec_ids, rec_distances, rec_indeces

In [12]:
usersdf = usersdf.merge(articlesdf[['contentId', 'title']], on='contentId', how='left')

In [13]:
contentId = -1999468346928419252
rec_ids, rec_distances, rec_indices = recommend(contentId, X, item_mapper, item_inv_mapper, k=10)

# Get the title of the selected contentId
selected_title = usersdf.loc[usersdf['contentId'] == contentId, 'title'].values[0]

# Create a DataFrame for the recommended items
rec_df = pd.DataFrame({'contentId': rec_ids, 'similarity': rec_distances})

# Merge the recommended items with the 'title' column from usersdf on 'contentId'
recommended_items = rec_df.merge(usersdf[['contentId', 'title']], on='contentId', how='left')

# Remove duplicate titles if they exist
recommended_items = recommended_items.drop_duplicates(subset='title').head(10)

# Print the message with the title of the selected article
print(f"If you like '{selected_title}', you may also enjoy:\n")
print(recommended_items['title'].to_list())  # Display a list of recommended titles


List of recommended item indeces:
[285 341 379 382 636  37 219 425 336 335 460]

List of recommended item IDs:
[np.int64(-548976469460064550), np.int64(372531153711028286), np.int64(428486625959995206), np.int64(6783622248311192269), np.int64(-8314629309720421219), np.int64(-3875881534863988868), np.int64(1580124381109623727), np.int64(-655909700262679295), np.int64(-667193404227875686), np.int64(2372438485070148864)]

List of recommended item similarity to selected item:
[0.36318471 0.41003838 0.50736517 0.52489637 0.54450977 0.54522713
 0.55040138 0.56891946 0.59150878 0.60723649]

If you like 'Docker - Build, Ship, and Run Any App, Anywhere', you may also enjoy:

['Using Honcho to Create a Multi-Process Docker Container', 'DevOps requer profundas mudanças operacionais que o CIO precisa promover - CIO', 'Most Interesting APIs in 2016: Cognitive Computing', 'Par de alianças impede que seu parceiro assista Netflix sem você', 'Hacked Website Report - 2016/Q3', 'Visa lança desafio para s

In [ ]:
# import joblib

# # Define the contentId, user_mapper, item_mapper, and X matrix
# contentId = 310515487419366995
# rec_ids, rec_distances, rec_indices = recommend(contentId, X, item_mapper, item_inv_mapper, k=10)

# # Save the entire recommender components
# joblib.dump({
#     'X': X,
#     'user_mapper': user_mapper,
#     'item_mapper': item_mapper,
#     'user_inv_mapper': user_inv_mapper,
#     'item_inv_mapper': item_inv_mapper
# }, 'collaborative_model.sav')

# print("Recommender model has been saved successfully!")


In [20]:
import pandas as pd

# Number of recommendations per movie
k = 5

# Initialize DataFrame with a column for the liked movie
df_recommendations = pd.DataFrame({'If you liked': list(item_mapper)})

# Add columns for recommendations
for i in range(1, k):
    df_recommendations[f'Recommendation {i}'] = None

# Populate recommendations
for index, row in df_recommendations.iterrows():
    contentId = row['If you liked']  # Movie ID stored in the column

    # Get a ranked list of recommendations
    rec_ids, rec_distances, rec_indeces = recommend(row[0], X, item_mapper, item_inv_mapper, k=k, messages=False)

    # Store the title of the liked movie
    df_recommendations.at[index, 'If you liked'] = articlesdf.loc[articlesdf['contentId'] == contentId, 'title'].values[0]

    # Store the titles of recommended movies
    for i, r in enumerate(rec_ids):
        df_recommendations.at[index, f'Recommendation {i+1}'] = articlesdf.loc[articlesdf['contentId'] == r, 'title'].values[0]

# Export recommendations to CSV
df_recommendations.to_csv('article_recommendations.csv', index=False)

print("CSV file 'article_recommendations.csv' has been saved.")


<ipython-input-20-f38d7294d857>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rec_ids, rec_distances, rec_indeces = recommend(row[0], X, item_mapper, item_inv_mapper, k=k, messages=False)
<ipython-input-20-f38d7294d857>:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Chrome OS now has Material Design for the desktop' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_recommendations.at[index, 'If you liked'] = articlesdf.loc[articlesdf['contentId'] == contentId, 'title'].values[0]
<ipython-input-20-f38d7294d857>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with 

CSV file 'article_recommendations.csv' has been saved.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


df = pd.read_csv('/Users/kyleebrown/Documents/IS 455/shared_articles.csv')
df_users = pd.read_csv('/Users/kyleebrown/Documents/IS 455/users_interactions.csv')

In [ ]:
print(df.shape)
df.isna().sum()

In [ ]:
df['authorUserAgent'].fillna('unknown', inplace=True)
df['authorRegion'].fillna('unknown', inplace=True)
df['authorCountry'].fillna('unknown', inplace=True)

In [ ]:
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()[:100]))  # Use only first 500 word

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer and Remove stopwords
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the data to a tfidf matrix
tfidf_matrix = tfidf.fit_transform(df['text'])

# Print the shape of the tfidf_matrix
print(tfidf_matrix.shape)

# Preview the matrix by placing it into a DataFrame (which we won't need later)
df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=df['text'])
df_tfidf.iloc[2221:2226]

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity between each movie description
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# For easier viewing, put it in a dataframe
pd.DataFrame(cosine_sim)


In [ ]:
import pandas as pd

def get_all_recommendations(df, sim_matrix, n=5, output_file="recommendations.csv"):
    all_recommendations = []

    for item_id in range(len(df)):
        if item_id >= len(sim_matrix):  # Ensure item_id is within bounds
            continue

        # Get similarity scores for the item
        sim_scores = list(enumerate(sim_matrix[item_id]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_similar = [pair for pair in sim_scores[1:n+1] if pair[1] < 1]  # Skip itself if score == 1

        # Store results in a structured format
        recommendations = [df.loc[rec_id, "title"] for rec_id, score in top_similar]
        while len(recommendations) < n:
            recommendations.append("")  # Ensure exactly n columns

        all_recommendations.append([df.loc[item_id, "title"]] + recommendations)

    # Convert to DataFrame and save to CSV
    column_names = ["Title"] + [f"Recommendation {i+1}" for i in range(n)]
    rec_df = pd.DataFrame(all_recommendations, columns=column_names)
    rec_df.to_csv(output_file, index=False)

    print(f"Final recommendations saved to {output_file}")
    return rec_df

# Run the function
recommendations_df = get_all_recommendations(df, cosine_sim)
